In [79]:
from PIL import Image
from keras.preprocessing import image
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input
import matplotlib.pyplot as plt
from keras_vggface.vggface import VGGFace
from sklearn.model_selection import train_test_split
import numpy as np
import glob
import pandas as pd
import csv

path = 'img_2102.bmp'
img = Image.open('Release/Data/Images/' + path ).convert('L')
max_edge = max(img.size[0], img.size[1])
print(img.size)
image_sum = img.size[0] + img.size[1]
black = Image.new('L',(max_edge, max_edge),0)
crop_img = [int(max_edge/2 - img.size[0]/2) , int(max_edge/2 - img.size[1]/2),
            int(max_edge/2 + (img.size[0]+1)/2) ,int(max_edge/2 + (img.size[1]+1)/2)]


if (image_sum + crop_img[0]+crop_img[1] != crop_img[2] +crop_img[3]):
    if (crop_img[0] == 0):
        black.paste(img, [crop_img[0],crop_img[1]+1, crop_img[2],crop_img[3]])
    else:
        black.paste(img, [crop_img[0]+1,crop_img[1], crop_img[2],crop_img[3]])
else:
    black.paste(img, [crop_img[0],crop_img[1], crop_img[2],crop_img[3]])

    img = black.resize((224,224))
img = np.array(img).astype(np.float32)

img -= np.mean(img)
imag /= np.std(img )




(138, 182)


In [ ]:
(380, 366)
[0, 7, 380, 373]

In [52]:
bmi_file = pd.read_csv("Release\Data\data.csv")
bmi_file['id'] = bmi_file.index
bmi = bmi_file['bmi']

i = 0
X_train_key = []
X_test_key  =[]
for index, row in bmi_file.iterrows():
    #print(row)
    t = row['name']
    if row['is_training'] == 1:
        
        X_train_key.append(t)
    else:
        X_test_key.append(t)

In [81]:
len(X_train_key[:4])

['img_0.bmp', 'img_1.bmp', 'img_2.bmp', 'img_3.bmp']

In [53]:
def get_data_by_id(path,bmi_file):
    
    img = Image.open('Release/Data/Images/' + path ).convert('L')
    #print(img)
    try:
        max_edge = max(img.size[0], img.size[1])
        black = Image.new('L',(max_edge, max_edge),0)
        black.paste(img, [int(max_edge/2 - img.size[0]/2),int(max_edge/2 - img.size[1]/2),
                            int(max_edge/2 + (img.size[0]+1)/2),
                            int(max_edge/2 + (img.size[1]+1)/2)] )

        img = black.resize((224,224))
        img = np.array(img).astype(np.float32)

        img -= np.mean(img)
        img /= np.std(img )

        
    except ValueError:
        print('error')
        img = Image.open('Release/Data/Images/' + path ).convert('L')
        img = img.resize((224,224))
        img = np.array(img).astype(np.float32)
        img -= np.mean(img)
        img /= np.std(img )
    #print(path)
    bmi = bmi_file[bmi_file['name'] == path]['bmi']
    #print (type(bmi))
    return img, bmi
image, bmi = get_data_by_id(path,bmi_file)
print(image.shape)

(224, 224)


In [54]:
id_list = ['0'] #list of id's
size = len(id_list)
image_batch = np.zeros((size,224,224,3))
bmi_batch = np.zeros((size))

for i in range(size):
    image = img
    bmi = 50
    image_batch[i,:,:,0] = image
    image_batch[i,:,:,1] = image
    image_batch[i,:,:,2] = image
    bmi_batch[i] = bmi

plt.imshow(image_batch[0,:,:,0], cmap = 'Greys_r')
print (image_batch.shape)

(1, 224, 224, 3)


In [8]:
base_model = VGG16(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)

X_train_key = ['0']
#X_test_key = ['232','233']
train_embeddings = np.zeros((len(X_train_key), 4096))
train_bmis = np.zeros((len(X_train_key)))

for i,key in enumerate(X_train_key):
    imag = image_batch
    x = np.expand_dims(imag, axis=0)
    x = preprocess_input(imag)
    print(x.shape)
    fc6_pool_features = model.predict(x)
    train_embeddings[i] = fc6_pool_features.flatten()
    train_bmis[i] = bmi_batch


(1, 224, 224, 3)


In [12]:
train_embeddings[0]

array([ 0.        ,  1.36069763,  0.        , ...,  0.        ,
        1.14239836,  0.62497115])

In [4]:
bmi_file = pd.read_csv("Release\Data\data.csv")
bmi_file = bmi_file.head(10)

bmi_file['id'] = bmi_file.index
bmi_file = bmi_file.drop('Unnamed: 0',axis = 1)
bmi = bmi_file['bmi']


In [5]:
bmi_file

,bmi,gender,is_training,name,id
0,34.207396,Male,1,img_0.bmp,0
1,26.453720,Male,1,img_1.bmp,1
2,34.967561,Female,1,img_2.bmp,2
3,22.044766,Female,1,img_3.bmp,3
4,37.758789,Female,1,img_4.bmp,4
5,24.886475,Female,1,img_5.bmp,5
6,25.845588,Female,1,img_6.bmp,6
7,21.740701,Female,1,img_7.bmp,7
8,30.615858,Female,1,img_8.bmp,8
9,26.788876,Female,1,img_9.bmp,9
